In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_json("../input/train.json")
train.loc[:, ['created']] = pd.to_datetime(train['created'])
test = pd.read_json("../input/test.json")
test.loc[:, ['created']] = pd.to_datetime(test['created'])
print(train.shape)
print(test.shape)
print(train.head())

In [ ]:
num_vars = ['bathrooms', 'bedrooms', 'created_month', 'created_day', 'latitude', 'longitude', 'photo_num', 'price']
cat_vars = ['building_id', 'display_address', 'manager_id', 'street_address']
other_vars = ['created', 'description', 'features', 'photos']

**data process**

In [ ]:
def process(data):
    # x = data[['bathrooms', 'bedrooms', 'latitude', 'longitude', 'price']]
    x = data[['latitude', 'longitude', 'price']]
    # x.loc[:, 'created'] = pd.to_datetime(data['created'])
    x.loc[:, 'created_month'] = data['created'].dt.month
    x.loc[:, 'created_day'] = data['created'].dt.day
    x.loc[:, 'created_hour'] = data['created'].dt.hour
    x.loc[:, 'description_len'] = data.photos.apply(len)
    x.loc[:, 'features_num'] = data.features.apply(len)
    x.loc[:, 'photos_num'] = data.photos.apply(len)
    data['bathrooms_x_bedrooms'] = data.apply(lambda x: '{}b{}b'.format(x['bathrooms'], x['bedrooms']), 1)
    x = x.add(pd.get_dummies(data['bathrooms_x_bedrooms'], sparse=True))
    if 'interest_level' in data:
        y = data['interest_level']
    else:
        y = None
    return x, y

**the categorical vars of train and test are highly coincide**

In [ ]:
var_temp = 'building_id'
test[var_temp].isin(train[var_temp]).sum()

In [ ]:
var_temp = 'manager_id'
test[var_temp].isin(train[var_temp]).sum()

**test the predictors and outcome**

In [ ]:
x_train, y_train = process(train)

In [ ]:

pd.crosstab(train['interest_level'], x_train['created_day']).apply(lambda s: s/sum(s))

In [ ]:
pd.crosstab(train['interest_level'], train['created'].dt.hour).apply(lambda s: s/sum(s))

In [ ]:
train.groupby(['interest_level', 'bathrooms'])['interest_level'].count()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, oob_score=True, verbose=1)
rf.fit(x_train, y_train)

In [ ]:
rf.oob_score_   

In [ ]:
rf.n_features_

In [ ]:
x_test, _ = process(test)

In [ ]:
y_test = rf.predict_proba(x_test)

In [ ]:
x_train.columns

In [ ]:
rf.feature_importances_ 

In [ ]:
rf.classes_ 

In [ ]:
 test["listing_id"].head()

In [ ]:
y_test_df = pd.DataFrame(y_test, columns=rf.classes_)
y_test_df["listing_id"] = test["listing_id"].tolist()
y_test_df.head()

In [ ]:
y_test_df.to_csv("submission_rf.csv", index=False)